# SemMed の DB を整備するコード



## ID テーブルと sentence テーブルの作成

* SemMed を解凍したデータを分割し、テーブルを作成している

In [8]:
import sqlite3
import csv

input_file = "/workspace/ssd4t/yoshikawa/0_DATA/SemMed/00_data_from_semmed/semmedVER43_2024_R_SENTENCE.csv"
db_output_path = "/workspace/ssd4t/yoshikawa/0_DATA/SemMed/01_semmed_db/"

# 1 つの DB に含む行数の上限
line_max = 50000

line_index = 1
file_seqno = 1

with open(input_file, "r", encoding="utf-8", errors="ignore") as in_file:
    reader = csv.reader(in_file)
    db_filename = f"{db_output_path}{file_seqno:04}_semmed.db"
    conn = sqlite3.connect(db_filename)
    curs = conn.cursor()

    # IDテーブルとSentenceテーブルを作成
    curs.execute("""
    CREATE TABLE IF NOT EXISTS id (
        sent_id INT,
        pmid INT
    );
    """)
    curs.execute("""
    CREATE TABLE IF NOT EXISTS sentence (
        sent_id INT,
        type TEXT,
        num INT,
        start INT,
        end INT,
        sentence TEXT
    );
    """)

    for row in reader:
        # 必要な列を抽出
        sent_id = row[0]
        pmid = row[1]
        type_ = row[2]
        num = row[3]
        start = row[4]
        sentence = row[5]
        end = row[6]

        # IDテーブルに挿入
        curs.execute("INSERT OR IGNORE INTO id (sent_id, pmid) VALUES (?, ?);", (sent_id, pmid))

        # Sentenceテーブルに挿入
        curs.execute("INSERT INTO sentence (sent_id, type, num, start, end, sentence) VALUES (?, ?, ?, ?, ?, ?);", (sent_id, type_, num, start, end, sentence))

        line_index += 1

        # 行数が上限に達した場合、新しいデータベースに切り替え
        if line_index > line_max:
            conn.commit()
            conn.close()

            line_index = 1
            file_seqno += 1

            # 新しいデータベースを開く
            db_filename = f"{db_output_path}{file_seqno:04}_semmed.db"
            conn = sqlite3.connect(db_filename)
            curs = conn.cursor()
            curs.execute("""
            CREATE TABLE IF NOT EXISTS id (
                sent_id INT,
                pmid INT
            );
            """)
            curs.execute("""
            CREATE TABLE IF NOT EXISTS sentence (
                sent_id INT,
                type TEXT,
                num INT,
                start INT,
                end INT,
                sentence TEXT
            );
            """)

    conn.commit()
    conn.close()

## pred_sent テーブルの作成

In [4]:
# predication と sentence の結合（id）

import sqlite3
from tqdm import tqdm

conn = sqlite3.connect('/workspace/ssd4t/yoshikawa/0_DATA/SemMed/02_semmed_pred/semmed_pred.db')
cursor = conn.cursor()

# インデックス作成
cursor.execute('CREATE INDEX IF NOT EXISTS idx_raw_sent_id ON raw(sent_id)')
conn.commit()
conn.close()

def combine_pred_sent(file_number):
    # メインのデータベースに接続
    conn = sqlite3.connect(f'/workspace/ssd4t/yoshikawa/0_DATA/SemMed/01_semmed_db/{file_number:04}_semmed.db')
    cursor = conn.cursor()

    # 別のデータベースをATTACHで追加
    cursor.execute("ATTACH DATABASE '/workspace/ssd4t/yoshikawa/0_DATA/SemMed/02_semmed_pred/semmed_pred.db' AS pred_db")

    # インデックスを作成して高速化
    cursor.execute('CREATE INDEX IF NOT EXISTS idx_id_sent_id ON id(sent_id)')

    cursor.execute("DROP TABLE IF EXISTS pred_sent")

    # 新しいテーブルを作成し、結合結果を挿入
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS pred_sent AS
        SELECT
            p.pred_id,
            s.sent_id
        FROM
            pred_db.raw p
        INNER JOIN
            id s
        ON
            p.sent_id = s.sent_id
    ''')

    # コミットしてデータベース接続を閉じる
    conn.commit()
    conn.close()

for i in tqdm(range(144, 145)):
    combine_pred_sent(i)

100%|██████████| 1/1 [00:11<00:00, 11.32s/it]


## predication テーブルの作成

In [2]:
# predication と sentence の結合（ predication の中身）

import sqlite3
from tqdm import tqdm

def combine_pred_sent(file_number):
    # メインのデータベースに接続
    conn = sqlite3.connect(f'/workspace/ssd4t/yoshikawa/0_DATA/SemMed/01_semmed_db/{file_number:04}_semmed.db')
    cursor = conn.cursor()

    # 別のデータベースをATTACHで追加
    cursor.execute("ATTACH DATABASE '/workspace/ssd4t/yoshikawa/0_DATA/SemMed/02_semmed_pred/semmed_pred.db' AS pred_db")

    cursor.execute("DROP TABLE IF EXISTS predication")

    # 新しいテーブルを作成し、結合結果を挿入
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS predication AS
        SELECT
            s.pred_id,
            p.predicate,
            p.sub_name,
            p.sub_ty,
            p.obj_name,
            p.obj_ty
        FROM
            pred_db.raw p
        INNER JOIN
            pred_sent s
        ON
            p.pred_id = s.pred_id
    ''')

    # コミットしてデータベース接続を閉じる
    conn.commit()
    conn.close()

for i in tqdm(range(38, 5265)):
    combine_pred_sent(i)

100%|██████████| 5227/5227 [01:45<00:00, 49.60it/s] 


## pred_aux テーブルの作成

In [8]:
def enable_wal_mode():
    conn = sqlite3.connect('/workspace/ssd4t/yoshikawa/0_DATA/SemMed/03_semmed_pred_aux/pred_aux.db')
    cursor = conn.cursor()
    cursor.execute('PRAGMA journal_mode=WAL;')
    conn.commit()
    conn.close()

enable_wal_mode()

In [16]:
# predication と aux の結合（id）

import sqlite3
from tqdm import tqdm

conn = sqlite3.connect('/workspace/ssd4t/yoshikawa/0_DATA/SemMed/03_semmed_pred_aux/pred_aux.db')
cursor = conn.cursor()

# インデックス作成
cursor.execute('CREATE INDEX IF NOT EXISTS idx_raw_pred_id ON raw(pred_id)')
conn.commit()
conn.close()

def combine_pred_sent(file_number):
    # メインのデータベースに接続
    conn = sqlite3.connect(f'/workspace/ssd4t/yoshikawa/0_DATA/SemMed/01_semmed_db/{file_number:04}_semmed.db')
    cursor = conn.cursor()

    # 別のデータベースをATTACHで追加
    cursor.execute("ATTACH DATABASE '/workspace/ssd4t/yoshikawa/0_DATA/SemMed/03_semmed_pred_aux/pred_aux.db' AS aux_db")

    # インデックスを作成して高速化
    cursor.execute('CREATE INDEX IF NOT EXISTS idx_pred_sent_pred_id ON pred_sent(pred_id)')

    cursor.execute("DROP TABLE IF EXISTS pred_aux")

    # 新しいテーブルを作成し、結合結果を挿入
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS pred_aux AS
        SELECT
            p.pred_aux_id,
            s.pred_id
        FROM
            aux_db.raw p
        INNER JOIN
            pred_sent s
        ON
            p.pred_id = s.pred_id
    ''')

    # コミットしてデータベース接続を閉じる
    conn.commit()
    conn.close()

for i in tqdm(range(3211, 5265)):
    combine_pred_sent(i)

100%|██████████| 2054/2054 [9:11:03<00:00, 16.10s/it]   


# aux テーブルの作成

In [17]:
# predication と aux の結合（ predication の中身）

import sqlite3
from tqdm import tqdm

def combine_pred_sent(file_number):
    # メインのデータベースに接続
    conn = sqlite3.connect(f'/workspace/ssd4t/yoshikawa/0_DATA/SemMed/01_semmed_db/{file_number:04}_semmed.db')
    cursor = conn.cursor()

    # 別のデータベースをATTACHで追加
    cursor.execute("ATTACH DATABASE '/workspace/ssd4t/yoshikawa/0_DATA/SemMed/03_semmed_pred_aux/pred_aux.db' AS aux_db")

    cursor.execute("DROP TABLE IF EXISTS aux")

    # 新しいテーブルを作成し、結合結果を挿入
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS aux AS
        SELECT
            s.pred_aux_id,
            p.sub_text,
            p.sub_dist,
            p.sub_maxdist,
            p.sub_start,
            p.sub_end,
            p.sub_score,
            p.ind_ty,
            p.pred_start,
            p.pred_end,
            p.obj_text,
            p.obj_dist,
            p.obj_maxdist,
            p.obj_start,
            p.obj_end,
            p.obj_score
        FROM
            aux_db.raw p
        INNER JOIN
            pred_aux s
        ON
            p.pred_id = s.pred_id
    ''')

    # コミットしてデータベース接続を閉じる
    conn.commit()
    conn.close()

for i in tqdm(range(3211, 5265)):
    combine_pred_sent(i)

100%|██████████| 2054/2054 [00:34<00:00, 58.94it/s]
